In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import copy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

#### Chuẩn bị dữ liệu

##### Các dữ liệu dùng để làm map tham chiếu

In [2]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'tt']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [3]:
#Tạo dict map thời gian và số lượng cổ phiếu
period_map = pd.read_excel("../period_data/period_stock_list.xlsx", sheet_name='period_map')
period_map_dict = period_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q3"
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
    else:
        quarter = "q4"
        previous_quarter = "q1"
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        if quarter == 'q4':
            return f'{previous_quarter}_{year-1}'
        else:
            return f'{previous_quarter}_{year}'
        
#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [period_map_dict['q2_2020'][0], period_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [period_map_dict[get_quarter('current_quarter')][0], period_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [period_map_dict[get_quarter('previous_quarter')][0], period_map_dict[get_quarter('previous_quarter')][1]]

#Lấy ra list cổ phiếu của giai đoạn hiện tại
period_stock_list = pd.read_excel("../period_data/period_stock_list.xlsx", sheet_name='period_stock_list')
curren_stock_list = period_stock_list[get_quarter('current_quarter')].dropna().tolist()

##### Các biến thời gian

In [4]:
#Khởi tạo ngày và thời gian hiện tại
today = pd.to_datetime(
        pd.read_csv('D:\\t2m-project\\ami-data\\ami_eod_data\\VNINDEX.csv')\
        .sort_values('date', ascending=False).reset_index(drop=True)\
        ['date'].iloc[0]
        , format='%y%m%d')

current_time = pd.to_datetime(
               pd.read_csv('D:\\t2m-project\\ami-data\\ami_itd_data\\HNXINDEX.csv')\
               .sort_values('date', ascending=False).reset_index(drop=True)\
               ['date'].iloc[0]
               , format='%y%m%d %H%M%S')

#Khởi tạo time_series bao gồm tất cả khung thời gian của ngày hiện tại
time_series_list = []
time_series_list.extend(pd.date_range(start=f'{today} 09:00:00', end=f'{today} 11:25:00', freq='5T'))
time_series_list.extend(pd.date_range(start=f'{today} 13:00:00', end=f'{today} 14:55:00', freq='5T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

#Khởi tạo khung thời gian bắt đầu từ 9h15 để vẽ các biểu đồ
itd_series = pd.DataFrame(time_series_list[3:]).rename(columns={0:'date'})

In [5]:
def calculate_time_percent(time):
    start_time_am = dt.time(9, 00)
    end_time_am = dt.time(11, 30)
    start_time_pm = dt.time(13, 00)
    end_time_pm = dt.time(15, 00)

    def time_difference_in_minutes(time1, time2):
        delta1 = dt.timedelta(hours=time1.hour, minutes=time1.minute, seconds=time1.second)
        delta2 = dt.timedelta(hours=time2.hour, minutes=time2.minute, seconds=time2.second)
        diff = delta2 - delta1
        return diff.seconds // 60

    time = (time + timedelta(minutes=5)).time()
    full_time_range = time_difference_in_minutes(start_time_am, end_time_am) + time_difference_in_minutes(start_time_pm, end_time_pm)

    if time <= end_time_am:
        time_range = time_difference_in_minutes(start_time_am, time)
    elif time >= start_time_pm:
        time_range = time_difference_in_minutes(start_time_am, time) - time_difference_in_minutes(end_time_am, start_time_pm)

    return time_range/full_time_range

#Điều chỉnh lại time_series bỏ đi các hàng thời gian chưa có dữ liệu
time_series = time_series.loc[time_series['date'] <= current_time].sort_values('date', ascending=False).reset_index(drop=True)

#Tính thêm time percent
time_percent = time_series.copy()
time_percent['percent'] = time_percent['date'].apply(calculate_time_percent)
time_percent['percent'] = time_percent['percent'].apply(lambda x: x if x < 1 else 1)

if len(time_percent['percent']) >= 1:
    current_time_percent = time_percent['percent'].iloc[0]
else: 
    current_time_percent = 1

In [6]:
#Tạo bảng thời gian update
def get_update_time(start_time_am, end_time_am, start_time_pm, end_time_pm):
    if (dt.datetime.now()).weekday() <= 4:
        current_time = dt.datetime.now().time()
        if current_time < start_time_am: current_time = end_time_pm
        elif (current_time >= start_time_am) & (current_time < end_time_am): current_time = current_time
        elif (current_time >= end_time_am) & (current_time < start_time_pm): current_time = end_time_am
        elif (current_time >= start_time_pm) & (current_time < end_time_pm): current_time = current_time
        elif current_time >= end_time_pm: current_time = end_time_pm
        return current_time
    if (dt.datetime.now()).weekday() > 4:
        return end_time_pm

time_update = get_update_time(dt.time(9, 00), dt.time(11, 30), dt.time(13, 00), dt.time(15, 00))
date_time_update = dt.datetime.combine(current_time.date(), time_update)
update_time = pd.DataFrame([f"Cập nhât: {date_time_update.strftime('%d/%m/%Y %H:%M:%S')}"]).rename(columns={0:'date'})

##### Các dữ liệu về cổ phiếu và index

In [7]:
#Đọc toàn bộ các file csv được xuất ra từ ami eod
eod_item_dict = {}
folder_path = 'D:\\t2m-project\\ami-data\\ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        eod_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in eod_item_dict.items():
    df['date'] = pd.to_datetime(df['date'], format='%y%m%d')
    eod_item_dict[item] = df

#Tạo bảng tổng hớp tất cả các item
eod_item_df = pd.DataFrame(list(eod_item_dict.keys())).rename(columns={0:'item'})
eod_item_df['len'] = eod_item_df['item'].apply(lambda x: len(x))
eod_item_df['last_2chars'] = eod_item_df['item'].str[-2:]
eod_item_df['first_4chars'] = eod_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
index_name_df = eod_item_df[(eod_item_df['len']>3) & (eod_item_df['len']!=6) & (eod_item_df['len']<10) & 
                (eod_item_df['item']!='0001')].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
                
eod_stock_dict = {k:v.drop(['option'], axis=1) for k,v in eod_item_dict.items()  if k in curren_stock_list}
eod_index_dict = {k:v.rename(columns={'option':'value'}).drop('cap', axis=1)
                for k,v in eod_item_dict.items() if k in index_name_df['item'].tolist()}

In [8]:
#Tính toán các đường trung bình và các đường MA
eod_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in eod_stock_dict.items()}

eod_stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in eod_stock_dict.items()
}

eod_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in eod_stock_dict.items()
}
eod_stock_dict = {k: v[(v['date'] >= calculate_time_span[0]) & (v['date'] <= calculate_time_span[1])].sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in eod_stock_dict.items()}

In [9]:
#Thêm cột tên period và số lượng cổ phiếu từng thời kì
def assign_period(x):
    for key, value in period_map_dict.items():
        if (x >= pd.Timestamp(value[0])) & (x <= pd.Timestamp(value[1])):
            return key

for stock, df in eod_stock_dict.items():
    df['period'] = df['date'].apply(assign_period)
    df['count'] = df['period'].apply(lambda x: period_map_dict[x][2])

#Tính hệ số thanh khoản và đổi lại cap của cổ phiếu thành cap trung bình trong 20 phiên
for df in eod_stock_dict.values():
    df['liquid_ratio'] = df['volume'] / (df['ma5_V'])
    df['liquid_ratio'].iloc[0] = df['volume'].iloc[0] / ((df['ma5_V']).iloc[0]*current_time_percent)
    df['cap'] = df['cap'][::-1].rolling(window=20).mean()[::-1]

#Tạo một date_series bao gồm khoảng ngày tính toán eod
date_series = pd.DataFrame(eod_stock_dict['REE']['date']).rename(columns={0:'date'})    

#### Phân nhóm cổ phiếu

In [10]:
full_stock_classification_df = pd.read_excel("../period_data/stock_classification.xlsx", sheet_name='stock_classification')
stock_classification_df = full_stock_classification_df[full_stock_classification_df['stock'].isin(curren_stock_list)].reset_index(drop=True)

price_arr = []
cap_arr = []
for stock, df in eod_stock_dict.items():
    df = df[df['date'] > current_quarter_span[0]]
    price_arr.append(df['close'].iloc[-1].item())
    cap_arr.append(df['cap'].iloc[-1].item())

vonhoa_classification_df = stock_classification_df.copy()
vonhoa_classification_df['price'] = price_arr
vonhoa_classification_df['cap'] = cap_arr

cap_coef = sum(cap_arr)/10000
vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
        ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
        else
('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
        ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
        else
('large' if x['cap']>=100*cap_coef
        else 'penny'
)), axis=1)

stock_classification_df = pd.concat([stock_classification_df, vonhoa_classification_df['marketcap_group']], axis=1)

In [11]:
# Convert DataFrame columns to dictionaries for quick access
stock_by_industry = stock_classification_df.set_index('stock')['industry_name'].to_dict()
stock_by_perform = stock_classification_df.set_index('stock')['industry_perform'].to_dict()
stock_by_marketcap = stock_classification_df.set_index('stock')['marketcap_group'].to_dict()

# Initialize dictionaries
eod_all_stock = {}
eod_industry_name = {}
eod_industry_perform = {}
eod_marketcap_group = {}

# Function to create mappings based on category
def create_mapping(stock_dict, category_dict):
    category_map = {}
    for category, stocks in category_dict.items():
        category_map[category] = {stock: stock_dict[stock] for stock in stocks if stock in stock_dict}
    return category_map

# Precompute unique categories and relevant stocks
unique_industries = np.unique(list(stock_by_industry.values()))
unique_performs = np.unique(list(stock_by_perform.values()))
unique_marketcaps = ['large', 'mid', 'small', 'penny']

# Mapping for all_stock
eod_all_stock['all_stock'] = {key: value for key, value in eod_stock_dict.items()}

# Mapping for industry
for industry in unique_industries:
    relevant_stocks = [stock for stock, ind in stock_by_industry.items() if ind == industry]
    eod_industry_name[industry] = {stock: eod_stock_dict[stock] for stock in relevant_stocks if stock in eod_stock_dict}

# Mapping for performance
for performance in unique_performs:
    relevant_stocks = [stock for stock, perf in stock_by_perform.items() if perf == performance]
    eod_industry_perform[performance] = {stock: eod_stock_dict[stock] for stock in relevant_stocks if stock in eod_stock_dict}

# Mapping for marketcap
for marketcap in unique_marketcaps:
    relevant_stocks = [stock for stock, mcap in stock_by_marketcap.items() if mcap == marketcap]
    eod_marketcap_group[marketcap] = {stock: eod_stock_dict[stock] for stock in relevant_stocks if stock in eod_stock_dict}


#### Điểm dòng tiền từng cổ phiếu

In [12]:
def score_calculation(row):
    try:
        return (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                (row['volume']*row['close']) / (row['ma5_prev'] * row['ma5_V'])) * 100 \
                + ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng
    except ZeroDivisionError:
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev'])/100 #Cộng thêm lượng này để tránh các trường hợp điểm dòng tiền bằng nhau gây trùng xếp hạng


In [13]:
#Tính điểm dòng tiền cho từng cổ phiếu
eod_score_dict = {}
for stock in eod_stock_dict.keys():

    #Lọc ra các cột cần sử dụng và chỉ lấy 40 phiên gần nhất để tính
    temp_df = eod_stock_dict[stock][['stock', 'date', 'period', 'count', 'open', 'high', 'low', 'close', 'volume', 'liquid_ratio', 'ma5', 'ma5_V']]

    #Tính điểm dòng tiền t0 và t5
    temp_df['ma5_prev'] = temp_df['ma5'].shift(-1)
    temp_df['close_prev'] = temp_df['close'].shift(-1)
    temp_df['t0_score'] = temp_df.apply(score_calculation, axis=1)
    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]

    #Gán lại temp_df cho dict
    eod_score_dict[stock] = temp_df

#### Điểm dòng tiền nhóm cổ phiếu

- Dòng tiền vào nhóm cổ phiếu EOD

In [14]:
#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
def adjust_score_by_breath(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

#Hàm điều chỉnh điểm dòng tiền của cổ phiếu tránh sự đột biến khi đóng góp vào nhóm chung
def adjust_score_for_smooth(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score

#Áp dụng hàm điều chỉnh điểm phía trên vào các nhóm cổ phiếu, việc này lặp lại nhiều lần cho tới khi triệt tiêu sự đột biến
def apply_smooth_score(score_dict, group_type, type_name):

    if type_name == 'itd':
        initial_score_df = time_series.copy()
    elif type_name == 'eod':
        initial_score_df = date_series.copy()

    if group_type == 'all_stock':
        key_list = all_stock_key_list
    elif group_type == 'industry_perform':
        key_list = [key for key, value in group_map_dict.items() if value == 'hs']
    elif group_type == 'marketcap_group':
        key_list = [key for key, value in group_map_dict.items() if value == 'cap'] 
    elif group_type == 'industry_name':
        key_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]

    for key in key_list:
        score_df = initial_score_df.copy()
        if group_type == 'all_stock':
            stock_list = stock_classification_df['stock'].tolist()
        else:
            stock_list = [stock for stock in stock_classification_df[stock_classification_df[group_type]==key]['stock'].dropna().tolist()]
        for stock in stock_list:
            try: score_df[stock] = score_dict[stock][f't0_score']
            except: pass

        max_percent = max(0.1, min(5*(1/len(stock_list)), 0.5))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for stock in stock_list:
                    score_df[stock] = score_df.iloc[:, 1:].apply(adjust_score_for_smooth, axis=1, args=(stock, max_percent, mark))
            if mark[0] == 0: break

        for stock in stock_list:
            try: score_dict[stock][f't0_{group_type}'] = score_df[stock]
            except: pass

In [15]:
#Thêm các cột dòng tiền đóng góp vào các nhóm cổ phiếu vào các dict period (đã loại bỏ đột biến)
for group_type in ['all_stock','industry_name','industry_perform','marketcap_group']:
    apply_smooth_score(eod_score_dict, group_type, 'eod')

In [16]:
#Tính độ rộng cho từng phiên phục vụ cho việc điều chỉnh điểm dòng tiền
temp_df = date_series.copy()
for stock, df in eod_score_dict.items():
    temp_df[stock] = eod_score_dict[stock]['t0_score']
temp_df.iloc[:,1:] = temp_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

eod_market_breath = date_series.copy()

industry_name_breadth_dict = {}
for key in eod_industry_name.keys():
    stock_list = stock_classification_df[stock_classification_df['industry_name']==key]['stock'].tolist()
    industry_name_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

industry_perform_breadth_dict = {}
for key in eod_industry_perform.keys():
    stock_list = stock_classification_df[stock_classification_df['industry_perform']==key]['stock'].tolist()
    industry_perform_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

marketcap_group_breadth_dict = {}
for key in eod_marketcap_group.keys():
    stock_list = stock_classification_df[stock_classification_df['marketcap_group']==key]['stock'].tolist()
    marketcap_group_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

all_stock_breadth_dict = {}
for key in eod_all_stock.keys():
    stock_list = stock_classification_df['stock'].tolist()
    all_stock_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
for stock, df in eod_score_dict.items():
    name_of_industry_name = stock_classification_df[stock_classification_df['stock']==stock]['industry_name'].item()
    name_of_industry_perform = stock_classification_df[stock_classification_df['stock']==stock]['industry_perform'].item()
    name_of_marketcap_group = stock_classification_df[stock_classification_df['stock']==stock]['marketcap_group'].item()

    df[f't0_industry_name'] = adjust_score_by_breath(df['t0_industry_name'], eod_market_breath[name_of_industry_name])
    df[f't0_industry_perform'] = adjust_score_by_breath(df['t0_industry_perform'], eod_market_breath[name_of_industry_perform])
    df[f't0_marketcap_group'] = adjust_score_by_breath(df['t0_marketcap_group'], eod_market_breath[name_of_marketcap_group])
    df[f't0_all_stock'] = adjust_score_by_breath(df['t0_all_stock'], eod_market_breath['all_stock'])

In [17]:
def mean_of_net_values(df):
    net_values = df  # Lọc ra các giá trị âm
    return net_values.mean(axis=1)

# Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df_net = date_series.copy()

# Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df[stock_classification_df['industry_name']==nganh]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df[stock_classification_df['industry_perform']==group]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[group] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    for stock in stock_classification_df[stock_classification_df['marketcap_group']==marketcap]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = mean_of_net_values(score_df.iloc[:, 1:])
    eod_group_score_df_net[marketcap] = score_df['total']

eod_group_score_df_net = eod_group_score_df_net.fillna(0)

In [18]:
def mean_of_negative_values(df, length):
    negative_values = df[df < 0]  # Lọc ra các giá trị âm
    return negative_values.sum(axis=1)/length

# Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df_negative = date_series.copy()

# Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_name']==nganh]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_perform']==group]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[group] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['marketcap_group']==marketcap]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_negative[marketcap] = score_df['total']

eod_group_score_df_negative = eod_group_score_df_negative.fillna(0)

In [19]:
def mean_of_positive_values(df, length):
    positive_values = df[df >= 0]  # Lọc ra các giá trị âm
    return positive_values.sum(axis=1)/length

# Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df_positive = date_series.copy()

# Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_name']==nganh]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[nganh] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['industry_perform']==group]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[group] = score_df['total']

# Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    temp_stock_list = stock_classification_df[stock_classification_df['marketcap_group']==marketcap]['stock']
    for stock in temp_stock_list:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
    eod_group_score_df_positive[marketcap] = score_df['total']

eod_group_score_df_positive = eod_group_score_df_positive.fillna(0)

#### Tính dữ liệu cho MS

In [20]:
def transform_ms(stock_group):
    stock_dict = copy.deepcopy(stock_group)

    # Prepare a base date DataFrame from date_series
    dates_df = pd.DataFrame(date_series['date'].tolist(), columns=['date'])
    
    for group_name, stocks in stock_dict.items():
        # Initialize a DataFrame for group trends
        group_trends = dates_df.copy()

        # Compute trends across stocks
        for trend in ['trend_5p', 'trend_20p', 'trend_60p', 'trend_120p', 'trend_240p', 'trend_480p']:
            # Concatenate all trend data for current trend across all stocks
            trend_data = pd.concat([stocks[stock][trend] for stock in stocks], axis=1)
            trend_data.fillna(0, inplace=True)
            
            # Calculate the sum and percent for the trend
            sum_trend = trend_data.sum(axis=1)
            percent_trend = sum_trend / len(stocks)
            
            # Add to group trends DataFrame
            group_trends[f'{trend}'] = percent_trend

        stock_dict[group_name] = group_trends[group_trends['date'] >= current_quarter_span[0]].sort_values('date', ascending=False)

    return stock_dict

In [21]:
#Tính toán các biểu đồ MS cho các nhóm cổ phiếu
all_stock_ms = transform_ms(eod_all_stock)
industry_name_ms = transform_ms(eod_industry_name)
industry_perform_ms = transform_ms(eod_industry_perform)
marketcap_group_ms = transform_ms(eod_marketcap_group)

#Gộp tất cả biểu đồ MS vào 1 bảng
temp_market_ms = pd.DataFrame()
for item in [all_stock_ms, industry_name_ms, industry_perform_ms, marketcap_group_ms]:
    for group, df in item.items():
        df['name'] = group
        temp_market_ms = pd.concat([temp_market_ms, df], axis=0)

#Lấy dữ liệu lịch sử MS đã tính toán
period_market_ms = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_market_ms_df')

#Ghép bảng dữ liệu lịch sử với dữ liệu của quý này
full_market_ms = pd.concat([temp_market_ms, period_market_ms], axis=0).sort_values('date', ascending=False).reset_index(drop=True)

#### Tính dữ liệu cho group price index

In [22]:
def calculate_total_change(stock_group, name, price_index_date_series):
    period_index_df = price_index_date_series.copy()

    for stock, df in stock_group[name].items():
        period_index_df[stock] = df['close']
        period_index_df[stock] = period_index_df[stock][::-1].pct_change()[::-1]

    period_index_df['total_change'] = period_index_df.iloc[:,1:].sum(axis=1)
    period_index_df['total_change'] = ((period_index_df['total_change']/len(stock_group[name]))*100)
    period_index_df['total_change'] = period_index_df['total_change']*10

    return period_index_df['total_change']

In [23]:
#Lấy dữ liệu lịch sử group price change đã tính toán
period_group_price_change = pd.read_excel("../period_data/period_processed_data.xlsx", sheet_name='full_group_price_change_df')

#Tính dữ liệu group price change của quý hiện tại
temp_group_price_change = date_series.copy()
for key in all_stock_key_list:
    temp_group_price_change[key] = calculate_total_change(eod_all_stock, key, date_series)

for key in industry_name_list:
    temp_group_price_change[key] = calculate_total_change(eod_industry_name, key, date_series)

for key in industry_perform_list:
    temp_group_price_change[key] = calculate_total_change(eod_industry_perform, key, date_series)

for key in marketcap_group_list:
    temp_group_price_change[key] = calculate_total_change(eod_marketcap_group, key, date_series)

temp_group_price_change = temp_group_price_change[temp_group_price_change['date'] >= current_quarter_span[0]]

#Ghép dữ liệu thay đổi index các nhóm cổ phiếu
group_price_index_df = pd.concat([temp_group_price_change, period_group_price_change]).sort_values('date', ascending=False).reset_index(drop=True)

for key in group_stock_key_list:
    group_price_index_df[key] = group_price_index_df[key][::-1].cumsum()[::-1] + 1000

#Thêm cột VNINDEX để tham chiếu
group_price_index_df = group_price_index_df.merge(
    eod_index_dict['VNINDEX'][['date', 'close']].rename(columns={'close': 'VNINDEX'}),
    on='date',
    how='left'
)

#### Các hàm tính toán sử dụng trong phân bổ vốn

In [110]:
def transform_value(x):
    if x < 0.2:
        return 0
    elif 0.2 <= x < 0.3:
        return 0.2
    elif 0.3 <= x < 0.5:
        return 0.4
    elif 0.5 <= x < 0.7:
        return 0.6
    elif 0.7 <= x < 0.8:
        return 0.8
    elif x >= 0.8:
        return 1

def round_portion_nhom(row):
    numbers = row.values.tolist()
    
    # Làm tròn từng số đến một chữ số thập phân và lưu lại phần dư
    rounded_numbers = [round(num, 1) for num in numbers]
    remainders = [num - round(num, 1) for num in numbers]

    # Tính tổng của các số đã làm tròn
    total = sum(rounded_numbers)

    # Tính chênh lệch so với 1
    difference = 1 - total

    # Điều chỉnh các số có phần dư lớn nhất
    if difference > 0:
        for _ in range(int(difference * 10)):
            index = remainders.index(max(remainders))
            rounded_numbers[index] += 0.1
            remainders[index] = 0
    elif difference < 0:
        for _ in range(int(-difference * 10)):
            index = remainders.index(min(remainders))
            rounded_numbers[index] -= 0.1
            remainders[index] = 0

    # Điều chỉnh nếu tổng vẫn không chính xác
    while round(sum(rounded_numbers), 1) != 1.0:
        current_sum = round(sum(rounded_numbers), 1)
        adjustment = round(1.0 - current_sum, 1)
        if adjustment > 0:
            index = remainders.index(max(remainders))
            rounded_numbers[index] += adjustment
        elif adjustment < 0:
            index = remainders.index(min(remainders))
            rounded_numbers[index] += adjustment
        break

    return pd.Series(rounded_numbers, index=row.index)

def calculate_final_portion(df):
    df['final_portion'] = df['portion_raw']
    for i in range(len(df) - 2, -1, -1):
        if df['portion_phase'].iloc[i] == 1:
            if df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == -1:
            if df['final_portion'].iloc[i+1] < df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == 2:
            if df['final_portion'].iloc[i+1] != df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        if df['portion_phase'].iloc[i] == -2:
            if df['final_portion'].iloc[i+1] != df['final_portion'].iloc[i]:
                df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
    return df['final_portion']

def adjust_portion_T3(df):
    for i in range(len(df) - 4, -1, -1):
        df['final_portion'] = df['final_portion']
        if (df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i+2]) & (df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i]):
            df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
        elif (df['final_portion'].iloc[i+2] > df['final_portion'].iloc[i+3]) & (df['final_portion'].iloc[i+1] > df['final_portion'].iloc[i]):
            df['final_portion'].iloc[i] = df['final_portion'].iloc[i+1]
    return df['final_portion']

def adjust_group_portion_T3(df, column_list):
    for i in range(len(df) - 4, -1, -1):
        if (df[column_list].iloc[i] != df[column_list].iloc[i + 1]).any() \
        & (df[column_list].iloc[i + 1] != df[column_list].iloc[i + 2]).any():
            df.loc[i, column_list] = df.loc[i + 1, column_list]
        elif (df[column_list].iloc[i] != df[column_list].iloc[i + 1]).any() \
        & (df[column_list].iloc[i + 2] != df[column_list].iloc[i + 3]).any():
            df.loc[i, column_list] = df.loc[i + 1, column_list]

    return df[column_list]

In [111]:
def round_to_nearest(value, base=0.2):
    """Làm tròn một giá trị tới bội số gần nhất của base."""
    return round(value / base) * base

def round_portion_nganh(row):
    numbers = row.values.tolist()
    
    # Bỏ qua các giá trị NaN
    numbers = [0 if pd.isna(num) else num for num in numbers]

    # Làm tròn từng số tới mốc gần nhất
    rounded_numbers = [round_to_nearest(num) for num in numbers]
    remainders = [num - round_to_nearest(num) for num in numbers]

    # Tính tổng của các số đã làm tròn
    total = sum(rounded_numbers)

    # Tính chênh lệch so với 1
    difference = 1 - total

    # Điều chỉnh các số có phần dư lớn nhất
    if difference > 0:
        for _ in range(int(difference / 0.2)):
            index = remainders.index(max(remainders))
            if rounded_numbers[index] < 1.0:
                rounded_numbers[index] += 0.2
                remainders[index] = 0
    elif difference < 0:
        for _ in range(int(-difference / 0.2)):
            index = remainders.index(min(remainders))
            if rounded_numbers[index] > 0.0:
                rounded_numbers[index] -= 0.2
                remainders[index] = 0

    # Điều chỉnh nếu tổng vẫn không chính xác
    while round(sum(rounded_numbers), 1) != 1.0:
        current_sum = round(sum(rounded_numbers), 1)
        adjustment = round(1.0 - current_sum, 1)
        if adjustment > 0:
            index = remainders.index(max(remainders))
            if rounded_numbers[index] < 1.0:
                rounded_numbers[index] += 0.2
        elif adjustment < 0:
            index = remainders.index(min(remainders))
            if rounded_numbers[index] > 0.0:
                rounded_numbers[index] -= 0.2
        break

    return pd.Series(rounded_numbers, index=row.index)

In [112]:
#TÍnh toán tín hiệu của MS
def calculate_ms(ms_df):
    index_ms = ms_df

    index_ms['5p_shift1'] = index_ms['trend_5p'].shift(-1)
    index_ms['5p_shift2'] = index_ms['trend_5p'].shift(-2)
    index_ms['5p_shift4'] = index_ms['trend_5p'].shift(-4)
    index_ms['20p_shift1'] = index_ms['trend_20p'].shift(-1)
    index_ms['20p_shift2'] = index_ms['trend_20p'].shift(-2)
    index_ms['20p_shift4'] = index_ms['trend_20p'].shift(-4)
    index_ms['60p_shift1'] = index_ms['trend_60p'].shift(-1)
    index_ms['60p_shift2'] = index_ms['trend_60p'].shift(-2)

    #Check điểm mua
    index_ms['5p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift1']) >= 0 else 0, axis = 1)
    index_ms['5p_upcheck4'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift4']) >= 0 else 0, axis = 1)
    index_ms['5p_upcheck'] = index_ms.apply(lambda x: 1 if (x['5p_upcheck1'] == 1) & (x['5p_upcheck4'] == 1) else 0, axis = 1)

    index_ms['20p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift1']) >= 0 else 0, axis = 1)
    index_ms['20p_upcheck4'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift4']) >= 0 else 0, axis = 1)
    index_ms['20p_upcheck'] = index_ms.apply(lambda x: 1 if (x['20p_upcheck1'] == 1) & (x['20p_upcheck4'] == 1) else 0, axis = 1)

    index_ms['60p_upcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_60p'] - x['60p_shift1']) >= 0 else 0, axis = 1)
    index_ms['60p_upcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_60p'] - x['60p_shift2']) >= 0 else 0, axis = 1)
    index_ms['60p_upcheck'] = index_ms.apply(lambda x: 1 if (x['60p_upcheck1'] == 1) & (x['60p_upcheck2'] == 1) else 0, axis = 1)

    index_ms['up_check'] = index_ms.apply(lambda x: 1 if (x['5p_upcheck'] == 1) & (x['20p_upcheck'] == 1) & (x['60p_upcheck'] == 1) & 
                                                        (x['trend_5p'] < 0.9) & (x['trend_20p'] < 0.9) &
                                                        ((x['trend_5p'] > 0.3) & (x['trend_20p'] > 0.2))

                                                   else (2 if (x['trend_5p'] >= 0.9) | (x['trend_20p'] >= 0.9) else 0)
                                                        , axis = 1)

    #Check điểm bán
    index_ms['5p_downcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift1']) < 0 else 0, axis = 1)
    index_ms['5p_downcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_5p'] - x['5p_shift2']) < 0 else 0, axis = 1)
    index_ms['5p_downcheck'] = index_ms.apply(lambda x: 1 if (x['5p_downcheck1'] == 1) & (x['5p_downcheck2'] == 1) else 0, axis = 1)

    index_ms['20p_downcheck1'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift1']) < 0 else 0, axis = 1)
    index_ms['20p_downcheck2'] = index_ms.apply(lambda x: 1 if (x['trend_20p'] - x['20p_shift2']) < 0 else 0, axis = 1)
    index_ms['20p_downcheck'] = index_ms.apply(lambda x: 1 if (x['20p_downcheck1'] == 1) & (x['20p_downcheck2'] == 1) else 0, axis = 1)

    index_ms['down_check'] = index_ms.apply(lambda x: 1 if (x['5p_downcheck'] == 1) & (x['20p_downcheck'] == 1) & 
                                                            ((x['trend_5p'] > 0.1) | (x['trend_20p'] > 0.1)) & 
                                                            ((x['trend_5p'] < 0.3) | ((x['trend_5p'] < 0.4) & (x['trend_20p'] < 0.6)))

                                                        else (2 if (x['trend_5p'] <= 0.1) & (x['trend_20p'] <= 0.1) else 0)
                                                        , axis = 1)

    return index_ms

#### Tính toán phân bổ vốn tổng

In [131]:
#Tính phân bổ vốn hoá tổng chưa điều chỉnh
phan_bo_von_raw = pd.DataFrame()
phan_bo_von_raw['date'] = date_series['date']
phan_bo_von_raw[['A+','B+','C+','D+']] = eod_group_score_df_positive[['A','B','C','D']][::-1].rolling(window=5, min_periods=1).mean()[::-1]
phan_bo_von_raw[['A-','B-','C-','D-']] = eod_group_score_df_negative[['A','B','C','D']][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

phan_bo_von_raw['A_raw'] = (phan_bo_von_raw['A+'] - phan_bo_von_raw['A-']) / (phan_bo_von_raw['A+'] + phan_bo_von_raw['A-'])
phan_bo_von_raw['B_raw'] = (phan_bo_von_raw['B+'] - phan_bo_von_raw['B-']) / (phan_bo_von_raw['B+'] + phan_bo_von_raw['B-'])
phan_bo_von_raw['C_raw'] = (phan_bo_von_raw['C+'] - phan_bo_von_raw['C-']) / (phan_bo_von_raw['C+'] + phan_bo_von_raw['C-'])
phan_bo_von_raw['D_raw'] = (phan_bo_von_raw['D+'] - phan_bo_von_raw['D-']) / (phan_bo_von_raw['D+'] + phan_bo_von_raw['D-'])
phan_bo_von_raw = phan_bo_von_raw.fillna(0)

phan_bo_von_raw['A_portion'] = phan_bo_von_raw['A_raw'].apply(lambda x: x*0.3 if x > 0 else 0)
phan_bo_von_raw['B_portion'] = phan_bo_von_raw['B_raw'].apply(lambda x: x*0.3 if x > 0 else 0)
phan_bo_von_raw['C_portion'] = phan_bo_von_raw['C_raw'].apply(lambda x: x*0.3 if x > 0 else 0)
phan_bo_von_raw['D_portion'] = phan_bo_von_raw['D_raw'].apply(lambda x: x*0.1 if x > 0 else 0)

phan_bo_von_raw['sum'] = phan_bo_von_raw[['A_portion','B_portion','C_portion','D_portion']].sum(axis=1).apply(lambda x: x if x >= 0.2 else 0)

In [132]:
#Điều chỉnh lại phân bổ vốn hoá tổng theo MS
index_ms = calculate_ms(full_market_ms[full_market_ms['name']=='all_stock'][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)

phan_bo_von_final = pd.DataFrame()
phan_bo_von_final[['date','A_portion_raw','B_portion_raw','C_portion_raw','D_portion_raw','portion_raw']] = phan_bo_von_raw[['date','A_portion','B_portion','C_portion','D_portion','sum']]

#Tạo các giai đoạn mua và bán
phan_bo_von_final = phan_bo_von_final.merge(index_ms[['date','up_check','down_check']], on='date', how='left')
phan_bo_von_final['portion_phase'] = phan_bo_von_final.apply(
    lambda x: 2 if x['up_check'] == 2 else 
            (1 if (x['up_check'] == 1) & (x['portion_raw'] > 0) else
            (-1 if x['down_check'] == 1 else
            (-2 if x['down_check'] == 2 else 
            None))), 
    axis=1)

#Thêm trạng thái đầu tiên là mua vào để khi fill ko bị NaN
phan_bo_von_final['portion_phase'].iloc[-1] = 1
phan_bo_von_final['portion_phase'] = phan_bo_von_final['portion_phase'].bfill()

#Điều chỉnh tỉ trọng cuối cùng theo các giai đoạn mua bán
phan_bo_von_final['final_portion'] = calculate_final_portion(phan_bo_von_final)

#Điều chỉnh để nếu có tăng giảm trong phân bổ tỉ tăng lên 1 và giảm về 0 luôn
phan_bo_von_final['final_portion'] = phan_bo_von_final.apply(lambda x: 1 if (x['portion_phase'] > 0) & (x['final_portion'] > 0)  else x['final_portion'], axis=1)
phan_bo_von_final['final_portion'] = phan_bo_von_final.apply(lambda x: 0 if (x['portion_phase'] < 0) & (x['final_portion'] < 1)  else x['final_portion'], axis=1)

#Điều chỉnh lại để tránh mua bán ko kịp T2
phan_bo_von_final['final_portion'] = adjust_portion_T3(phan_bo_von_final)

In [133]:
#Tạo dict để điều chỉnh tỉ trọng tăng giảm các nhóm vốn hoá theo biểu đồ MS
phan_bo_von_dict = {}

for nhom in ['A','B','C','D']:
    phan_bo_von_dict[nhom] = pd.DataFrame()
    phan_bo_von_dict[nhom]['index'] = group_price_index_df[nhom]
    phan_bo_von_dict[nhom][['date','portion_raw']] = phan_bo_von_raw[['date', nhom + '_portion']]

    temp_ms = calculate_ms(full_market_ms[full_market_ms['name']==nhom][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)
    phan_bo_von_dict[nhom] = phan_bo_von_dict[nhom].merge(temp_ms[['date','up_check','down_check']], on='date', how='left')

    #Tạo các giai đoạn mua và bán
    phan_bo_von_dict[nhom]['portion_phase'] = phan_bo_von_dict[nhom].apply(
        lambda x: 2 if x['up_check'] == 2 else 
                (1 if x['up_check'] == 1 else
                (-1 if x['down_check'] == 1 else
                (-2 if x['down_check'] == 2 else 
                None))), 
        axis=1)
    phan_bo_von_dict[nhom]['portion_phase'] = phan_bo_von_dict[nhom]['portion_phase'].bfill()
    phan_bo_von_dict[nhom]['final_portion'] = calculate_final_portion(phan_bo_von_dict[nhom])
    phan_bo_von_dict[nhom]['final_portion'] = adjust_portion_T3(phan_bo_von_dict[nhom])

#Thêm các cột điều chỉnh tỉ trọng cho các nhóm vốn hoá
for nhom in ['A','B','C','D']:
    phan_bo_von_final[nhom + '_portion'] = phan_bo_von_dict[nhom]['final_portion']
for nhom in ['A','B','C','D']:
    phan_bo_von_final[nhom + '_portion'] = phan_bo_von_dict[nhom]['final_portion']/phan_bo_von_final[[item + '_portion' for item in ['A','B','C','D']]].sum(axis=1)

# Làm tròn tỉ trọng với tổng luôn bằng 1
phan_bo_von_final[[item + '_portion' for item in ['A','B','C','D']]] = phan_bo_von_final[[item + '_portion' for item in ['A','B','C','D']]].apply(round_portion_nhom, axis=1)
# FIll lại 0 cho các giá trị NaN khi chạy hàm round_portion_nhom
phan_bo_von_final = phan_bo_von_final.fillna(0)

#Điều chỉnh tỉ trọng lại cho các nhóm vốn hoá nếu tỉ trọng tổng bằng 0 thì tất cả cũng bằng
for column in [item + '_portion' for item in ['A','B','C','D']]:
    phan_bo_von_final[column] = phan_bo_von_final.apply(lambda x: 0 if x['final_portion'] == 0 else x[column], axis = 1)

#Điều chỉnh để tỉ trọng ko thay đổi 2 phiên liên tiếp
phan_bo_von_final[['A_portion', 'B_portion', 'C_portion', 'D_portion']] = adjust_group_portion_T3(phan_bo_von_final, ['A_portion', 'B_portion', 'C_portion', 'D_portion'])

#### Phân bổ vốn theo ngành

In [134]:
def calculate_group_total_change(row, nganh_list):
    total_change = 0
    for nganh in nganh_list:
        total_change += row[nganh + '_portion'] * row[nganh + '_index_change']

    return total_change

##### Ngành hiệu suất A

In [135]:
nganh_hsA_list = ['ban_le','bds','chung_khoan','tai_chinh','thep','vlxd', 'xd']

phan_bo_von_hsA_raw = pd.DataFrame()
phan_bo_von_hsA_raw['date'] = date_series['date']

phan_bo_von_hsA_raw[[item + '+' for item in nganh_hsA_list]] = eod_group_score_df_positive[nganh_hsA_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
phan_bo_von_hsA_raw[[item + '-' for item in nganh_hsA_list]] = eod_group_score_df_negative[nganh_hsA_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

for column in nganh_hsA_list:
    phan_bo_von_hsA_raw[f'{column}_raw'] = (phan_bo_von_hsA_raw[f'{column}+'] - phan_bo_von_hsA_raw[f'{column}-']) / (phan_bo_von_hsA_raw[f'{column}+'] + phan_bo_von_hsA_raw[f'{column}-'])
phan_bo_von_hsA_raw = phan_bo_von_hsA_raw.fillna(0)

phan_bo_von_hsA_raw['ban_le_portion'] = phan_bo_von_hsA_raw['ban_le_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsA_raw['bds_portion'] = phan_bo_von_hsA_raw['bds_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsA_raw['chung_khoan_portion'] = phan_bo_von_hsA_raw['chung_khoan_raw'].apply(lambda x: x*0.2 if x > 0 else 0)
phan_bo_von_hsA_raw['tai_chinh_portion'] = phan_bo_von_hsA_raw['tai_chinh_raw'].apply(lambda x: x*0.1 if x > 0 else 0)
phan_bo_von_hsA_raw['thep_portion'] = phan_bo_von_hsA_raw['thep_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsA_raw['vlxd_portion'] = phan_bo_von_hsA_raw['vlxd_raw'].apply(lambda x: x*0.1 if x > 0 else 0)
phan_bo_von_hsA_raw['xd_portion'] = phan_bo_von_hsA_raw['xd_raw'].apply(lambda x: x*0.15 if x > 0 else 0)

phan_bo_von_hsA_raw = phan_bo_von_hsA_raw[['date'] + [item + '_portion' for item in nganh_hsA_list]]

In [136]:
phan_bo_von_hsA_dict = {}

for nganh in nganh_hsA_list:
    phan_bo_von_hsA_dict[nganh] = pd.DataFrame()
    phan_bo_von_hsA_dict[nganh]['index'] = group_price_index_df[nganh]
    phan_bo_von_hsA_dict[nganh][['date','portion_raw']] = phan_bo_von_hsA_raw[['date', nganh + '_portion']]

    temp_ms = calculate_ms(full_market_ms[full_market_ms['name']==nganh][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)
    phan_bo_von_hsA_dict[nganh] = phan_bo_von_hsA_dict[nganh].merge(temp_ms[['date','up_check','down_check']], on='date', how='left')

    #Tạo các giai đoạn mua và bán
    phan_bo_von_hsA_dict[nganh]['portion_phase'] = phan_bo_von_hsA_dict[nganh].apply(
        lambda x: 2 if x['up_check'] == 2 else 
                (1 if x['up_check'] == 1 else
                (-1 if x['down_check'] == 1 else
                (-2 if x['down_check'] == 2 else 
                None))), 
        axis=1)
    phan_bo_von_hsA_dict[nganh]['portion_phase'] = phan_bo_von_hsA_dict[nganh]['portion_phase'].bfill()
    
    phan_bo_von_hsA_dict[nganh]['final_portion'] = calculate_final_portion(phan_bo_von_hsA_dict[nganh])

In [137]:
phan_bo_von_hsA_final = pd.DataFrame()
phan_bo_von_hsA_final['date'] = phan_bo_von_hsA_raw['date']
phan_bo_von_hsA_final['total_portion'] = phan_bo_von_final['A_portion']

#Tính toán portion của từng ngành trên tổng
for nganh in nganh_hsA_list:
    phan_bo_von_hsA_final[nganh + '_portion_raw'] = phan_bo_von_hsA_dict[nganh]['final_portion']
for nganh in nganh_hsA_list:
    phan_bo_von_hsA_final[nganh + '_portion'] = phan_bo_von_hsA_dict[nganh]['final_portion']/phan_bo_von_hsA_final[[item + '_portion_raw' for item in nganh_hsA_list]].sum(axis=1)
phan_bo_von_hsA_final[[item + '_portion' for item in nganh_hsA_list]] = phan_bo_von_hsA_final[[item + '_portion' for item in nganh_hsA_list]].apply(round_portion_nganh, axis=1)

#Tính toán sự thay đổi index
for nganh in nganh_hsA_list:
    phan_bo_von_hsA_final[nganh + '_index'] = group_price_index_df[nganh]
for nganh in nganh_hsA_list:
    phan_bo_von_hsA_final[nganh + '_index_change'] = phan_bo_von_hsA_final[nganh + '_index'][::-1].pct_change()[::-1]

phan_bo_von_hsA_final = phan_bo_von_hsA_final.fillna(0)

#Điều chỉnh tỉ trọng để nếu tỉ trọng tổng bằng 0 thì các tỉ trọng khác cũng bằng 0
phan_bo_von_hsA_final.loc[phan_bo_von_hsA_final['total_portion'] == 0, [item + '_portion' for item in nganh_hsA_list]] = 0

#Điều chỉnh để tỉ trọng ko thay đổi 2 phiên liên tiếp
phan_bo_von_hsA_final[[item + '_portion' for item in nganh_hsA_list]] = adjust_group_portion_T3(phan_bo_von_hsA_final, [item + '_portion' for item in nganh_hsA_list])

#Tính thay đổi của index tổng của nhóm hiệu suất theo tỉ trọng của từng nhóm ngành
phan_bo_von_hsA_final['total_index_change'] = phan_bo_von_hsA_final.apply(lambda row: calculate_group_total_change(row, nganh_hsA_list), axis=1)

#Lọc ra các cột cần thiết
filtered_columns = [col for col in phan_bo_von_hsA_final.columns if col.endswith('_portion') or col.endswith('_index_change') or col == 'date']
phan_bo_von_hsA_final = phan_bo_von_hsA_final[filtered_columns]

##### Ngành hiệu suất B

In [138]:
nganh_hsB_list = ['det_may','cong_nghiep','hoa_chat','dau_khi','thuy_san','khoang_san']

phan_bo_von_hsB_raw = pd.DataFrame()
phan_bo_von_hsB_raw['date'] = date_series['date']

phan_bo_von_hsB_raw[[item + '+' for item in nganh_hsB_list]] = eod_group_score_df_positive[nganh_hsB_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
phan_bo_von_hsB_raw[[item + '-' for item in nganh_hsB_list]] = eod_group_score_df_negative[nganh_hsB_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

for column in nganh_hsB_list:
    phan_bo_von_hsB_raw[f'{column}_raw'] = (phan_bo_von_hsB_raw[f'{column}+'] - phan_bo_von_hsB_raw[f'{column}-']) / (phan_bo_von_hsB_raw[f'{column}+'] + phan_bo_von_hsB_raw[f'{column}-'])

phan_bo_von_hsB_raw['det_may_portion'] = phan_bo_von_hsB_raw['det_may_raw'].apply(lambda x: x*0.2 if x > 0 else 0)
phan_bo_von_hsB_raw['cong_nghiep_portion'] = phan_bo_von_hsB_raw['cong_nghiep_raw'].apply(lambda x: x*0.1 if x > 0 else 0)
phan_bo_von_hsB_raw['hoa_chat_portion'] = phan_bo_von_hsB_raw['hoa_chat_raw'].apply(lambda x: x*0.2 if x > 0 else 0)
phan_bo_von_hsB_raw['dau_khi_portion'] = phan_bo_von_hsB_raw['dau_khi_raw'].apply(lambda x: x*0.2 if x > 0 else 0)
phan_bo_von_hsB_raw['thuy_san_portion'] = phan_bo_von_hsB_raw['thuy_san_raw'].apply(lambda x: x*0.2 if x > 0 else 0)
phan_bo_von_hsB_raw['khoang_san_portion'] = phan_bo_von_hsB_raw['khoang_san_raw'].apply(lambda x: x*0.1 if x > 0 else 0)

phan_bo_von_hsB_raw = phan_bo_von_hsB_raw[['date'] + [item + '_portion' for item in nganh_hsB_list]]

In [139]:
phan_bo_von_hsB_dict = {}

for nganh in nganh_hsB_list:
    phan_bo_von_hsB_dict[nganh] = pd.DataFrame()
    phan_bo_von_hsB_dict[nganh]['index'] = group_price_index_df[nganh]
    phan_bo_von_hsB_dict[nganh][['date','portion_raw']] = phan_bo_von_hsB_raw[['date', nganh + '_portion']]

    temp_ms = calculate_ms(full_market_ms[full_market_ms['name']==nganh][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)
    phan_bo_von_hsB_dict[nganh] = phan_bo_von_hsB_dict[nganh].merge(temp_ms[['date','up_check','down_check']], on='date', how='left')

    #Tạo các giai đoạn mua và bán
    phan_bo_von_hsB_dict[nganh]['portion_phase'] = phan_bo_von_hsB_dict[nganh].apply(
        lambda x: 2 if x['up_check'] == 2 else 
                (1 if x['up_check'] == 1 else
                (-1 if x['down_check'] == 1 else
                (-2 if x['down_check'] == 2 else 
                None))), 
        axis=1)
    phan_bo_von_hsB_dict[nganh]['portion_phase'] = phan_bo_von_hsB_dict[nganh]['portion_phase'].bfill()
    
    phan_bo_von_hsB_dict[nganh]['final_portion'] = calculate_final_portion(phan_bo_von_hsB_dict[nganh])

In [140]:
phan_bo_von_hsB_final = pd.DataFrame()
phan_bo_von_hsB_final['date'] = phan_bo_von_hsB_raw['date']
phan_bo_von_hsB_final['total_portion'] = phan_bo_von_final['B_portion']

#Tính toán portion của từng ngành trên tổng
for nganh in nganh_hsB_list:
    phan_bo_von_hsB_final[nganh + '_portion_raw'] = phan_bo_von_hsB_dict[nganh]['final_portion']
for nganh in nganh_hsB_list:
    phan_bo_von_hsB_final[nganh + '_portion'] = phan_bo_von_hsB_dict[nganh]['final_portion']/phan_bo_von_hsB_final[[item + '_portion_raw' for item in nganh_hsB_list]].sum(axis=1)
phan_bo_von_hsB_final[[item + '_portion' for item in nganh_hsB_list]] = phan_bo_von_hsB_final[[item + '_portion' for item in nganh_hsB_list]].apply(round_portion_nganh, axis=1)

#Tính toán sự thay đổi index
for nganh in nganh_hsB_list:
    phan_bo_von_hsB_final[nganh + '_index'] = group_price_index_df[nganh]
for nganh in nganh_hsB_list:
    phan_bo_von_hsB_final[nganh + '_index_change'] = phan_bo_von_hsB_final[nganh + '_index'][::-1].pct_change()[::-1]

phan_bo_von_hsB_final = phan_bo_von_hsB_final.fillna(0)

#Điều chỉnh tỉ trọng để nếu tỉ trọng tổng bằng 0 thì các tỉ trọng khác cũng bằng 0
phan_bo_von_hsB_final.loc[phan_bo_von_hsB_final['total_portion'] == 0, [item + '_portion' for item in nganh_hsB_list]] = 0

#Điều chỉnh để tỉ trọng ko thay đổi 2 phiên liên tiếp
phan_bo_von_hsB_final[[item + '_portion' for item in nganh_hsB_list]] = adjust_group_portion_T3(phan_bo_von_hsB_final, [item + '_portion' for item in nganh_hsB_list])

#Tính thay đổi của index tổng của nhóm hiệu suất theo tỉ trọng của từng nhóm ngành
phan_bo_von_hsB_final['total_index_change'] = phan_bo_von_hsB_final.apply(lambda row: calculate_group_total_change(row, nganh_hsB_list), axis=1)

#Lọc ra các cột cần thiết
filtered_columns = [col for col in phan_bo_von_hsB_final.columns if col.endswith('_portion') or col.endswith('_index_change') or col == 'date']
phan_bo_von_hsB_final = phan_bo_von_hsB_final[filtered_columns]

##### Ngành hiệu suất C

In [141]:
nganh_hsC_list = ['bds_kcn','thuc_pham','van_tai','cong_nghe','htd','ngan_hang']

phan_bo_von_hsC_raw = pd.DataFrame()
phan_bo_von_hsC_raw['date'] = date_series['date']

phan_bo_von_hsC_raw[[item + '+' for item in nganh_hsC_list]] = eod_group_score_df_positive[nganh_hsC_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
phan_bo_von_hsC_raw[[item + '-' for item in nganh_hsC_list]] = eod_group_score_df_negative[nganh_hsC_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

for column in nganh_hsC_list:
    phan_bo_von_hsC_raw[f'{column}_raw'] = (phan_bo_von_hsC_raw[f'{column}+'] - phan_bo_von_hsC_raw[f'{column}-']) / (phan_bo_von_hsC_raw[f'{column}+'] + phan_bo_von_hsC_raw[f'{column}-'])

phan_bo_von_hsC_raw['bds_kcn_portion'] = phan_bo_von_hsC_raw['bds_kcn_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsC_raw['thuc_pham_portion'] = phan_bo_von_hsC_raw['thuc_pham_raw'].apply(lambda x: x*0.20 if x > 0 else 0)
phan_bo_von_hsC_raw['van_tai_portion'] = phan_bo_von_hsC_raw['van_tai_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsC_raw['cong_nghe_portion'] = phan_bo_von_hsC_raw['cong_nghe_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsC_raw['htd_portion'] = phan_bo_von_hsC_raw['htd_raw'].apply(lambda x: x*0.15 if x > 0 else 0)
phan_bo_von_hsC_raw['ngan_hang_portion'] = phan_bo_von_hsC_raw['ngan_hang_raw'].apply(lambda x: x*0.20 if x > 0 else 0)

phan_bo_von_hsC_raw = phan_bo_von_hsC_raw[['date'] + [item + '_portion' for item in nganh_hsC_list]]

In [142]:
phan_bo_von_hsC_dict = {}

for nganh in nganh_hsC_list:
    phan_bo_von_hsC_dict[nganh] = pd.DataFrame()
    phan_bo_von_hsC_dict[nganh]['index'] = group_price_index_df[nganh]
    phan_bo_von_hsC_dict[nganh][['date','portion_raw']] = phan_bo_von_hsC_raw[['date', nganh + '_portion']]

    temp_ms = calculate_ms(full_market_ms[full_market_ms['name']==nganh][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)
    phan_bo_von_hsC_dict[nganh] = phan_bo_von_hsC_dict[nganh].merge(temp_ms[['date','up_check','down_check']], on='date', how='left')

    #Tạo các giai đoạn mua và bán
    phan_bo_von_hsC_dict[nganh]['portion_phase'] = phan_bo_von_hsC_dict[nganh].apply(
        lambda x: 2 if x['up_check'] == 2 else 
                (1 if x['up_check'] == 1 else
                (-1 if x['down_check'] == 1 else
                (-2 if x['down_check'] == 2 else 
                None))), 
        axis=1)
    phan_bo_von_hsC_dict[nganh]['portion_phase'] = phan_bo_von_hsC_dict[nganh]['portion_phase'].bfill()
    
    phan_bo_von_hsC_dict[nganh]['final_portion'] = calculate_final_portion(phan_bo_von_hsC_dict[nganh])

In [143]:
phan_bo_von_hsC_final = pd.DataFrame()
phan_bo_von_hsC_final['date'] = phan_bo_von_hsC_raw['date']
phan_bo_von_hsC_final['total_portion'] = phan_bo_von_final['C_portion']

#Tính toán portion của từng ngành trên tổng
for nganh in nganh_hsC_list:
    phan_bo_von_hsC_final[nganh + '_portion_raw'] = phan_bo_von_hsC_dict[nganh]['final_portion']
for nganh in nganh_hsC_list:
    phan_bo_von_hsC_final[nganh + '_portion'] = phan_bo_von_hsC_dict[nganh]['final_portion']/phan_bo_von_hsC_final[[item + '_portion_raw' for item in nganh_hsC_list]].sum(axis=1)
phan_bo_von_hsC_final[[item + '_portion' for item in nganh_hsC_list]] = phan_bo_von_hsC_final[[item + '_portion' for item in nganh_hsC_list]].apply(round_portion_nganh, axis=1)

#Tính toán sự thay đổi index
for nganh in nganh_hsC_list:
    phan_bo_von_hsC_final[nganh + '_index'] = group_price_index_df[nganh]
for nganh in nganh_hsC_list:
    phan_bo_von_hsC_final[nganh + '_index_change'] = phan_bo_von_hsC_final[nganh + '_index'][::-1].pct_change()[::-1]

phan_bo_von_hsC_final = phan_bo_von_hsC_final.fillna(0)

#Điều chỉnh tỉ trọng để nếu tỉ trọng tổng bằng 0 thì các tỉ trọng khác cũng bằng 0
phan_bo_von_hsC_final.loc[phan_bo_von_hsC_final['total_portion'] == 0, [item + '_portion' for item in nganh_hsC_list]] = 0

#Điều chỉnh để tỉ trọng ko thay đổi 2 phiên liên tiếp
phan_bo_von_hsC_final[[item + '_portion' for item in nganh_hsC_list]] = adjust_group_portion_T3(phan_bo_von_hsC_final, [item + '_portion' for item in nganh_hsC_list])

#Tính thay đổi của index tổng của nhóm hiệu suất theo tỉ trọng của từng nhóm ngành
phan_bo_von_hsC_final['total_index_change'] = phan_bo_von_hsC_final.apply(lambda row: calculate_group_total_change(row, nganh_hsC_list), axis=1)

#Lọc ra các cột cần thiết
filtered_columns = [col for col in phan_bo_von_hsC_final.columns if col.endswith('_portion') or col.endswith('_index_change') or col == 'date']
phan_bo_von_hsC_final = phan_bo_von_hsC_final[filtered_columns]

##### Ngành hiệu suất D

In [144]:
nganh_hsD_list = ['bao_hiem','dv_hatang','y_te','dulich_dv']

phan_bo_von_hsD_raw = pd.DataFrame()
phan_bo_von_hsD_raw['date'] = date_series['date']

phan_bo_von_hsD_raw[[item + '+' for item in nganh_hsD_list]] = eod_group_score_df_positive[nganh_hsD_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
phan_bo_von_hsD_raw[[item + '-' for item in nganh_hsD_list]] = eod_group_score_df_negative[nganh_hsD_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

for column in nganh_hsD_list:
    phan_bo_von_hsD_raw[f'{column}_raw'] = (phan_bo_von_hsD_raw[f'{column}+'] - phan_bo_von_hsD_raw[f'{column}-']) / (phan_bo_von_hsD_raw[f'{column}+'] + phan_bo_von_hsD_raw[f'{column}-'])

phan_bo_von_hsD_raw['bao_hiem_portion'] = phan_bo_von_hsD_raw['bao_hiem_raw'].apply(lambda x: x*0.25 if x > 0 else 0)
phan_bo_von_hsD_raw['dv_hatang_portion'] = phan_bo_von_hsD_raw['dv_hatang_raw'].apply(lambda x: x*0.25 if x > 0 else 0)
phan_bo_von_hsD_raw['y_te_portion'] = phan_bo_von_hsD_raw['y_te_raw'].apply(lambda x: x*0.25 if x > 0 else 0)
phan_bo_von_hsD_raw['dulich_dv_portion'] = phan_bo_von_hsD_raw['dulich_dv_raw'].apply(lambda x: x*0.25 if x > 0 else 0)

phan_bo_von_hsD_raw = phan_bo_von_hsD_raw[['date'] + [item + '_portion' for item in nganh_hsD_list]]

In [145]:
phan_bo_von_hsD_dict = {}

for nganh in nganh_hsD_list:
    phan_bo_von_hsD_dict[nganh] = pd.DataFrame()
    phan_bo_von_hsD_dict[nganh]['index'] = group_price_index_df[nganh]
    phan_bo_von_hsD_dict[nganh][['date','portion_raw']] = phan_bo_von_hsD_raw[['date', nganh + '_portion']]

    temp_ms = calculate_ms(full_market_ms[full_market_ms['name']==nganh][['date','trend_5p','trend_20p','trend_60p']]).sort_values('date', ascending=False).reset_index(drop=True)
    phan_bo_von_hsD_dict[nganh] = phan_bo_von_hsD_dict[nganh].merge(temp_ms[['date','up_check','down_check']], on='date', how='left')

    #Tạo các giai đoạn mua và bán
    phan_bo_von_hsD_dict[nganh]['portion_phase'] = phan_bo_von_hsD_dict[nganh].apply(
        lambda x: 2 if x['up_check'] == 2 else 
                (1 if x['up_check'] == 1 else
                (-1 if x['down_check'] == 1 else
                (-2 if x['down_check'] == 2 else 
                None))), 
        axis=1)
    phan_bo_von_hsD_dict[nganh]['portion_phase'] = phan_bo_von_hsD_dict[nganh]['portion_phase'].bfill()
    
    phan_bo_von_hsD_dict[nganh]['final_portion'] = calculate_final_portion(phan_bo_von_hsD_dict[nganh])

In [146]:
phan_bo_von_hsD_final = pd.DataFrame()
phan_bo_von_hsD_final['date'] = phan_bo_von_hsD_raw['date']
phan_bo_von_hsD_final['total_portion'] = phan_bo_von_final['D_portion']

#Tính toán portion của từng ngành trên tổng
for nganh in nganh_hsD_list:
    phan_bo_von_hsD_final[nganh + '_portion_raw'] = phan_bo_von_hsD_dict[nganh]['final_portion']
for nganh in nganh_hsD_list:
    phan_bo_von_hsD_final[nganh + '_portion'] = phan_bo_von_hsD_dict[nganh]['final_portion']/phan_bo_von_hsD_final[[item + '_portion_raw' for item in nganh_hsD_list]].sum(axis=1)
phan_bo_von_hsD_final[[item + '_portion' for item in nganh_hsD_list]] = phan_bo_von_hsD_final[[item + '_portion' for item in nganh_hsD_list]].apply(round_portion_nganh, axis=1)

#Tính toán sự thay đổi index
for nganh in nganh_hsD_list:
    phan_bo_von_hsD_final[nganh + '_index'] = group_price_index_df[nganh]
for nganh in nganh_hsD_list:
    phan_bo_von_hsD_final[nganh + '_index_change'] = phan_bo_von_hsD_final[nganh + '_index'][::-1].pct_change()[::-1]

phan_bo_von_hsD_final = phan_bo_von_hsD_final.fillna(0)

#Điều chỉnh tỉ trọng để nếu tỉ trọng tổng bằng 0 thì các tỉ trọng khác cũng bằng 0
phan_bo_von_hsD_final.loc[phan_bo_von_hsD_final['total_portion'] == 0, [item + '_portion' for item in nganh_hsD_list]] = 0

#Điều chỉnh để tỉ trọng ko thay đổi 2 phiên liên tiếp
phan_bo_von_hsD_final[[item + '_portion' for item in nganh_hsD_list]] = adjust_group_portion_T3(phan_bo_von_hsD_final, [item + '_portion' for item in nganh_hsD_list])

#Tính thay đổi của index tổng của nhóm hiệu suất theo tỉ trọng của từng nhóm ngành
phan_bo_von_hsD_final['total_index_change'] = phan_bo_von_hsD_final.apply(lambda row: calculate_group_total_change(row, nganh_hsD_list), axis=1)

#Lọc ra các cột cần thiết
filtered_columns = [col for col in phan_bo_von_hsD_final.columns if col.endswith('_portion') or col.endswith('_index_change') or col == 'date']
phan_bo_von_hsD_final = phan_bo_von_hsD_final[filtered_columns]

#### Backtest hiệu năng

In [147]:
def calculate_profit(df, profit_column, price_change_column):
    # Tạo bản sao của DataFrame để tránh sửa đổi trực tiếp
    df_copy = df.copy()
    
    # Tính toán sự biến động của lợi nhuận
    for i in range(len(df_copy) - 2, -1, -1):
        current_money = df_copy.loc[df_copy.index[i + 1], profit_column]
        if pd.isna(df_copy.loc[df_copy.index[i + 1], 'final_portion']):
            df_copy.loc[df_copy.index[i], profit_column] = current_money
            continue
        invested_money = df_copy.loc[df_copy.index[i + 1], 'final_portion'] * current_money
        non_invested_money = current_money - invested_money
        if pd.notna(df_copy.loc[df_copy.index[i], price_change_column]):
            invested_money *= (1 + df_copy.loc[df_copy.index[i], price_change_column])
        df_copy.loc[df_copy.index[i], profit_column] = invested_money + non_invested_money
    
    return df_copy

In [149]:
phan_bo_von_backtest = copy.deepcopy(phan_bo_von_final)
phan_bo_von_backtest = phan_bo_von_backtest.dropna()

phan_bo_von_backtest = phan_bo_von_final[phan_bo_von_final['date'] >= '2022-11-22']
# phan_bo_von_backtest = phan_bo_von_final[phan_bo_von_final['date'] >= '2023-10-30']

#total_index_change được tính theo tỉ trọng của từng ngành vào nhóm vốn hoá, sau đó ở bước này ta lại nhân với tỉ trọng từng nhóm vốn hoá
phan_bo_von_backtest['pct_nganh_index'] = phan_bo_von_hsA_final['total_index_change'] * phan_bo_von_backtest['A_portion']\
                                        + phan_bo_von_hsB_final['total_index_change'] * phan_bo_von_backtest['B_portion']\
                                        + phan_bo_von_hsC_final['total_index_change'] * phan_bo_von_backtest['C_portion']\
                                        + phan_bo_von_hsD_final['total_index_change'] * phan_bo_von_backtest['D_portion']

#Lấy cột VNINDEX để tính % thay đổi của VNINDEX từng phiên
phan_bo_von_backtest['vnindex'] = group_price_index_df['VNINDEX']
phan_bo_von_backtest['pct_vnindex'] = phan_bo_von_backtest['vnindex'][::-1].pct_change()[::-1]

#Lấy cột all_stock_index để tính % thay đổi của all_stock_index từng phiên
phan_bo_von_backtest['t2m_index'] = group_price_index_df['all_stock']
phan_bo_von_backtest['pct_t2m_index'] = phan_bo_von_backtest['t2m_index'][::-1].pct_change()[::-1]


phan_bo_von_backtest = phan_bo_von_backtest.dropna()

# Khởi tạo giá trị cho cột money ban đầu
initial_money = phan_bo_von_backtest['vnindex'].iloc[-1]
phan_bo_von_backtest['money_vnindex'] = 0
phan_bo_von_backtest['money_vnindex'].iloc[-1] = initial_money

phan_bo_von_backtest['money_t2m_index'] = phan_bo_von_backtest['money_vnindex']
phan_bo_von_backtest['money_nganh_index'] = phan_bo_von_backtest['money_vnindex']

# Áp dụng hàm tính toán
phan_bo_von_backtest = calculate_profit(phan_bo_von_backtest, 'money_vnindex', 'pct_vnindex')
phan_bo_von_backtest = calculate_profit(phan_bo_von_backtest, 'money_t2m_index', 'pct_t2m_index')
phan_bo_von_backtest = calculate_profit(phan_bo_von_backtest, 'money_nganh_index', 'pct_nganh_index')

phan_bo_von_backtest = phan_bo_von_backtest.dropna()

with pd.ExcelWriter('test_data.xlsx', engine='openpyxl') as writer:
    full_market_ms[full_market_ms['name']=='all_stock'].to_excel(writer, sheet_name='full_market_ms', index=False)
    phan_bo_von_backtest.to_excel(writer, sheet_name='phan_bo_von_backtest', index=False)
    phan_bo_von_hsA_final.to_excel(writer, sheet_name='phan_bo_von_hsA_final', index=False)
    phan_bo_von_hsB_final.to_excel(writer, sheet_name='phan_bo_von_hsB_final', index=False)
    phan_bo_von_hsC_final.to_excel(writer, sheet_name='phan_bo_von_hsC_final', index=False)
    phan_bo_von_hsD_final.to_excel(writer, sheet_name='phan_bo_von_hsD_final', index=False)

with pd.ExcelWriter(f"history/{datetime.now().date().strftime('%d_%m_%Y')}.xlsx", engine='openpyxl') as writer:
    phan_bo_von_backtest.to_excel(writer, sheet_name='phan_bo_von_backtest', index=False)
    full_market_ms[full_market_ms['name']=='all_stock'].to_excel(writer, sheet_name='full_market_ms', index=False)
    phan_bo_von_hsA_final.to_excel(writer, sheet_name='phan_bo_von_hsA_final', index=False)
    phan_bo_von_hsB_final.to_excel(writer, sheet_name='phan_bo_von_hsB_final', index=False)
    phan_bo_von_hsC_final.to_excel(writer, sheet_name='phan_bo_von_hsC_final', index=False)
    phan_bo_von_hsD_final.to_excel(writer, sheet_name='phan_bo_von_hsD_final', index=False)